# Market Anomaly Detection - Explainability Demo

This notebook demonstrates the explainability features of our market anomaly detection system.
We'll show how to use SHAP, LIME, attention mechanisms, and rule-based explanations.

In [ ]:
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import our explainability modules
from src.explainability.interface import explain_instance, pretty_print_explanation
from src.explainability.visualization import (
    plot_shap_summary, plot_lime_explanation, plot_attention_heatmap,
    create_explanation_dashboard, plot_feature_importance_comparison
)
from src.explainability.rule_based import MarketAnomalyRules

# Import data and model utilities
from src.data.synthetic_market_data import SyntheticMarketDataGenerator
from src.data.preprocessing import MarketDataPreprocessor

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports successful!")

## 1. Generate Sample Data

First, let's generate some synthetic market data with anomalies.

In [ ]:
# Generate synthetic market data
data_generator = SyntheticMarketDataGenerator()
market_data = data_generator.generate_market_session(
    n_samples=1000,
    anomaly_probability=0.1,
    random_state=42
)

print(f"Generated {len(market_data)} market data points")
print(f"Columns: {list(market_data.columns)}")

# Show sample data
market_data.head()

In [ ]:
# Preprocess the data
preprocessor = MarketDataPreprocessor()
processed_data = preprocessor.preprocess_data(market_data)

print(f"Processed data shape: {processed_data.shape}")
print(f"Feature names: {preprocessor.feature_names}")

# Split into normal and anomalous samples
anomaly_mask = market_data['is_anomaly'].values
normal_samples = processed_data[~anomaly_mask]
anomalous_samples = processed_data[anomaly_mask]

print(f"Normal samples: {len(normal_samples)}")
print(f"Anomalous samples: {len(anomalous_samples)}")

## 2. Train a Simple Model

Let's train a simple model for demonstration purposes.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Prepare training data
X = processed_data
y = anomaly_mask.astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print("Model Performance:")
print(classification_report(y_test, y_pred))

print(f"\nModel trained on {len(X_train)} samples")
print(f"Test accuracy: {model.score(X_test, y_test):.3f}")

## 3. SHAP Explanations

Let's use SHAP to explain model predictions.

In [ ]:
# Select some test samples for explanation
test_samples = X_test[:5]  # First 5 test samples
feature_names = preprocessor.feature_names

print("Running SHAP explanation...")
shap_explanation = explain_instance(
    model=model,
    observation=test_samples,
    method='shap',
    feature_names=feature_names,
    nsamples=50,  # Reduced for demo speed
    cache=False
)

print(pretty_print_explanation(shap_explanation))

In [ ]:
# Visualize SHAP results
if 'shap_values' in shap_explanation:
    plt.figure(figsize=(12, 8))
    plot_meta = plot_shap_summary(shap_explanation, figsize=(12, 8))
    plt.show()
    
    print(f"SHAP visualization metadata: {plot_meta}")
else:
    print("SHAP values not available for visualization")

## 4. LIME Explanations

Now let's try LIME explanations.

In [ ]:
print("Running LIME explanation...")
lime_explanation = explain_instance(
    model=model,
    observation=test_samples[:3],  # First 3 samples for LIME
    method='lime',
    feature_names=feature_names,
    num_features=8,
    cache=False
)

print(pretty_print_explanation(lime_explanation))

In [ ]:
# Visualize LIME results
if 'explanations' in lime_explanation and lime_explanation['explanations']:
    plt.figure(figsize=(12, 6))
    plot_meta = plot_lime_explanation(lime_explanation, instance_idx=0, figsize=(12, 6))
    plt.show()
    
    print(f"LIME visualization metadata: {plot_meta}")
else:
    print("LIME explanations not available for visualization")

## 5. Rule-Based Explanations

Let's use our rule-based system to explain anomalies.

In [ ]:
# Initialize rule-based system
rule_system = MarketAnomalyRules()

# Show rule system summary
rule_summary = rule_system.get_rule_summary()
print("Rule System Summary:")
print(f"Total rules: {rule_summary['total_rules']}")
print(f"Rule types: {rule_summary['rule_types']}")
print(f"Rule names: {rule_summary['rule_names']}")
print(f"Thresholds: {rule_summary['thresholds']}")

In [ ]:
# Test rule-based explanations on some samples
print("Testing rule-based explanations...")

# Test on a normal sample
normal_sample = normal_samples[0]
normal_explanation = rule_system.explain_observation(normal_sample, feature_names)

print("\n--- Normal Sample ---")
print(f"Anomaly score: {normal_explanation['anomaly_score']:.3f}")
print(f"Triggered rules: {normal_explanation['triggered_rules']}")
print(f"Explanation: {normal_explanation['explanation_text']}")

# Test on an anomalous sample
if len(anomalous_samples) > 0:
    anomalous_sample = anomalous_samples[0]
    anomalous_explanation = rule_system.explain_observation(anomalous_sample, feature_names)
    
    print("\n--- Anomalous Sample ---")
    print(f"Anomaly score: {anomalous_explanation['anomaly_score']:.3f}")
    print(f"Triggered rules: {anomalous_explanation['triggered_rules']}")
    print(f"Explanation: {anomalous_explanation['explanation_text']}")
else:
    print("\nNo anomalous samples available for testing")

In [ ]:
# Run rule-based explanation through the unified interface
print("Running rule-based explanation through unified interface...")
rule_explanation = explain_instance(
    model=model,  # Model not used for rule-based, but required by interface
    observation=test_samples[0],
    method='rule',
    feature_names=feature_names,
    cache=False
)

print(pretty_print_explanation(rule_explanation))

## 6. Comparison Dashboard

Let's create a comprehensive dashboard comparing all explanation methods.

In [ ]:
# Collect all explanations
all_explanations = []

if 'shap_values' in shap_explanation:
    all_explanations.append(shap_explanation)

if 'explanations' in lime_explanation:
    all_explanations.append(lime_explanation)

all_explanations.append(rule_explanation)

print(f"Creating dashboard with {len(all_explanations)} explanations...")

# Create dashboard
dashboard_dir = "../artifacts/explanations/demo_dashboard"
dashboard_data = create_explanation_dashboard(
    all_explanations,
    save_dir=dashboard_dir,
    title="Market Anomaly Detection - Explainability Demo"
)

print(f"Dashboard created in {dashboard_dir}")
print(f"Summary: {dashboard_data['summary']}")

In [ ]:
# Create feature importance comparison
if len(all_explanations) > 1:
    plt.figure(figsize=(14, 8))
    comparison_meta = plot_feature_importance_comparison(
        all_explanations,
        figsize=(14, 8)
    )
    plt.show()
    
    print(f"Feature importance comparison: {comparison_meta}")
else:
    print("Not enough explanations for comparison plot")

## 7. Interactive Analysis

Let's create some interactive analysis tools.

In [ ]:
# Analyze multiple samples and compare explanations
def analyze_sample(sample_idx):
    """Analyze a specific sample with all explanation methods."""
    sample = X_test[sample_idx:sample_idx+1]
    true_label = y_test[sample_idx]
    pred_label = model.predict(sample)[0]
    pred_proba = model.predict_proba(sample)[0]
    
    print(f"\n=== Sample {sample_idx} Analysis ===")
    print(f"True label: {true_label} ({'Anomaly' if true_label else 'Normal'})")
    print(f"Predicted: {pred_label} ({'Anomaly' if pred_label else 'Normal'})")
    print(f"Prediction confidence: {pred_proba[pred_label]:.3f}")
    
    # Rule-based explanation (fastest)
    rule_exp = explain_instance(
        model=model,
        observation=sample[0],
        method='rule',
        feature_names=feature_names,
        cache=False
    )
    
    print("\n--- Rule-based Explanation ---")
    print(f"Anomaly score: {rule_exp.get('anomaly_score', 0):.3f}")
    print(f"Triggered rules: {len(rule_exp.get('triggered_rules', []))}")
    if rule_exp.get('explanation_text'):
        print(f"Explanation: {rule_exp['explanation_text']}")
    
    return {
        'sample_idx': sample_idx,
        'true_label': true_label,
        'pred_label': pred_label,
        'confidence': pred_proba[pred_label],
        'rule_explanation': rule_exp
    }

# Analyze a few samples
sample_analyses = []
for i in [0, 5, 10, 15, 20]:  # Analyze samples at different indices
    if i < len(X_test):
        analysis = analyze_sample(i)
        sample_analyses.append(analysis)

In [ ]:
# Create summary statistics
print("\n=== Summary Statistics ===")

# Model performance summary
correct_predictions = sum(1 for a in sample_analyses if a['true_label'] == a['pred_label'])
print(f"Correct predictions: {correct_predictions}/{len(sample_analyses)}")

# Rule-based anomaly scores
rule_scores = [a['rule_explanation'].get('anomaly_score', 0) for a in sample_analyses]
print(f"Rule-based anomaly scores: min={min(rule_scores):.3f}, max={max(rule_scores):.3f}, mean={np.mean(rule_scores):.3f}")

# Most common triggered rules
all_triggered_rules = []
for a in sample_analyses:
    all_triggered_rules.extend(a['rule_explanation'].get('triggered_rules', []))

if all_triggered_rules:
    from collections import Counter
    rule_counts = Counter(all_triggered_rules)
    print(f"Most common triggered rules: {rule_counts.most_common(3)}")
else:
    print("No rules were triggered in the analyzed samples")

## 8. Explainability Insights

Let's summarize the insights from our explainability analysis.

In [ ]:
print("\n=== Explainability Insights ===")

# Feature importance insights
if 'shap_values' in shap_explanation:
    shap_values = np.array(shap_explanation['shap_values'])
    if shap_values.ndim == 2:
        feature_importance = np.mean(np.abs(shap_values), axis=0)
        top_features_idx = np.argsort(feature_importance)[-5:][::-1]
        
        print("\nTop 5 most important features (SHAP):")
        for i, idx in enumerate(top_features_idx):
            if idx < len(feature_names):
                print(f"  {i+1}. {feature_names[idx]}: {feature_importance[idx]:.4f}")

# Rule-based insights
rule_summary = rule_system.get_rule_summary()
print(f"\nRule-based system has {rule_summary['total_rules']} rules")
print(f"Rule types: {rule_summary['rule_types']}")

# Method comparison
print("\n=== Method Comparison ===")
print("SHAP:")
print("  + Provides feature importance with directionality")
print("  + Model-agnostic")
print("  - Computationally expensive")
print("  - Requires background data")

print("\nLIME:")
print("  + Local explanations for individual instances")
print("  + Model-agnostic")
print("  - Can be unstable")
print("  - Requires careful parameter tuning")

print("\nRule-based:")
print("  + Fast and interpretable")
print("  + Domain knowledge integration")
print("  + Consistent explanations")
print("  - Requires manual rule definition")
print("  - May miss complex patterns")

## 9. Save Results

Let's save our analysis results for future reference.

In [ ]:
# Save explanation results
results_dir = Path("../artifacts/explanations/demo_results")
results_dir.mkdir(parents=True, exist_ok=True)

# Save individual explanations
import json

if 'shap_values' in shap_explanation:
    with open(results_dir / "shap_explanation.json", 'w') as f:
        json.dump(shap_explanation, f, indent=2, default=str)
    print("✅ SHAP explanation saved")

if 'explanations' in lime_explanation:
    with open(results_dir / "lime_explanation.json", 'w') as f:
        json.dump(lime_explanation, f, indent=2, default=str)
    print("✅ LIME explanation saved")

with open(results_dir / "rule_explanation.json", 'w') as f:
    json.dump(rule_explanation, f, indent=2, default=str)
print("✅ Rule-based explanation saved")

# Save sample analyses
with open(results_dir / "sample_analyses.json", 'w') as f:
    json.dump(sample_analyses, f, indent=2, default=str)
print("✅ Sample analyses saved")

# Save model and preprocessor for later use
import pickle

with open(results_dir / "demo_model.pkl", 'wb') as f:
    pickle.dump(model, f)
print("✅ Model saved")

with open(results_dir / "preprocessor.pkl", 'wb') as f:
    pickle.dump(preprocessor, f)
print("✅ Preprocessor saved")

print(f"\n📁 All results saved to {results_dir}")

## Conclusion

This demo showcased the comprehensive explainability features of our market anomaly detection system:

1. **SHAP explanations** provide global feature importance and local explanations
2. **LIME explanations** offer instance-specific interpretations
3. **Rule-based explanations** give fast, interpretable insights based on domain knowledge
4. **Visualization tools** help understand and compare different explanation methods
5. **Unified interface** makes it easy to switch between explanation methods

The system provides multiple perspectives on model behavior, helping users understand:
- Which features are most important for anomaly detection
- Why specific instances were classified as anomalies
- How different explanation methods complement each other
- Domain-specific insights through rule-based analysis

This multi-faceted approach to explainability is crucial for building trust in AI systems used for financial market analysis.